In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
sys.path.insert(0, "../src/")
import importlib
from util import ipca
from sklearn.metrics import r2_score
importlib.reload(ipca)

<module 'util.ipca' from '/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/notebooks/../src/util/ipca.py'>

In [2]:
crsp_features = pd.read_csv("../data/ipca_features_final.csv")
crsp_features.head()

,Unnamed: 0.1,Unnamed: 0,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,...,gvkey,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq
0,0,0,02742010,60409,20092,0,2,7392.0,1990-01-31,0.25000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,02742010,60409,20092,0,2,7392.0,1990-02-28,0.25000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,02742010,60409,20092,0,2,7392.0,1990-03-30,-0.28125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,02742010,60409,20092,0,2,7392.0,1990-04-30,0.25000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,02742010,60409,20092,0,2,7392.0,1990-05-31,0.18750,...,1506.0,5.126,0.017,4.172,0.0,0.0,0.0,3.133,0.084,12.737


In [3]:
crsp_features['date'] = pd.to_datetime(crsp_features['date'] )

In [4]:
crsp_features.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'cusip', 'permno', 'permco', 'issuno',
       'hexcd', 'hsiccd', 'date', 'bidlo', 'askhi', 'prc', 'vol', 'ret', 'bid',
       'ask', 'shrout', 'cfacpr', 'cfacshr', 'altprc', 'spread', 'altprcdt',
       'retx', 'date.1', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn',
       'spindx', 'totval', 'totcnt', 'usdval', 'usdcnt', 'ticker', 'mktcap',
       'turnover', 'adjprc', 'momentum', 'short_term_reversal',
       'long_term_reversal', '52wk_high', 'price_rel_52wk_high', 'rf', 'beta',
       'idvol', 'unexp_vol', 'gvkey', 'revtq', 'niq', 'atq', 'epsfxq', 'cheq',
       'dlttq', 'cogsq', 'oibdpq', 'cshoq'],
      dtype='object')

## Test Train Split (Cross Sectional Test) 

In [5]:
all_stocks = pd.Series(crsp_features['permno'].unique())
all_stocks_eos = crsp_features[crsp_features['date']==pd.to_datetime('2024-12-31')]['permno']
test_stocks = all_stocks_eos.sample(int(len(all_stocks_eos)*0.2),random_state=27)
train_stocks = all_stocks[~all_stocks.isin(test_stocks)]
print(f"Length of train stocks {len(train_stocks)}")
print(f"Length of test stocks {len(test_stocks)}")

Length of train stocks 12016
Length of test stocks 1076


In [6]:
crsp_features['mkt_to_book'] = crsp_features['mktcap']/crsp_features['atq']

In [7]:
y_column = ['ret']
x_columns = ['vol','beta','mktcap','momentum','short_term_reversal','52wk_high','price_rel_52wk_high','idvol','unexp_vol','turnover','mkt_to_book','revtq', 'niq', 'atq', 'epsfxq', 'cheq',
       'dlttq', 'cogsq', 'oibdpq', 'cshoq']
crsp_features = crsp_features[x_columns+y_column+['date','permno']]

In [8]:
fund_features = ['revtq', 'niq', 'atq', 'epsfxq', 'cheq',
       'dlttq', 'cogsq', 'oibdpq', 'cshoq']


In [9]:
def rankNormalize(group):
    # Get all columns
    all_columns = list(group.columns)
    
    # Identify columns to exclude from normalization
    exclude_columns = ['permno', 'date','ret']
    
    # Get columns to normalize
    columns_to_normalize = [col for col in all_columns if col not in exclude_columns]
    
    # Create a copy of the original dataframe
    result = group.copy()
    
    # Apply rank normalization only to the columns that need it
    result[columns_to_normalize] = (
        group[columns_to_normalize].rank(axis=0, method='average') / 
        (group[columns_to_normalize].count())
    ) - 0.5
    
    # Return the dataframe with all columns
    return result

In [10]:
result = crsp_features.groupby('date').apply(rankNormalize).reset_index(level=0, drop=True)

/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_13606/2380117743.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = crsp_features.groupby('date').apply(rankNormalize).reset_index(level=0, drop=True)


In [11]:
result.head()

,vol,beta,mktcap,momentum,short_term_reversal,52wk_high,price_rel_52wk_high,idvol,unexp_vol,turnover,...,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,ret,date,permno
0,-0.260891,NaN,-0.368812,NaN,NaN,NaN,NaN,NaN,NaN,-0.421287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.166667,1990-01-31,60409
391,0.485149,NaN,0.437129,NaN,0.061848,NaN,NaN,NaN,NaN,0.447525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.157667,1990-01-31,24643
866,0.405198,NaN,-0.018812,NaN,-0.253105,NaN,NaN,NaN,NaN,0.403960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.058824,1990-01-31,64151
1438,0.325248,NaN,0.002475,NaN,-0.456284,NaN,NaN,NaN,NaN,0.429208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.128527,1990-01-31,14403
1795,0.319802,NaN,0.179703,NaN,-0.161451,NaN,NaN,NaN,NaN,0.179208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.171429,1990-01-31,63845


In [12]:
def get_nan_count(return_series):
        return return_series.isna().sum()
result.groupby('permno')['revtq'].apply(get_nan_count)

permno
10001      0
10002    129
10009    128
10011      0
10012      3
        ... 
93431      0
93432      2
93433      0
93434      2
93436      2
Name: revtq, Length: 13092, dtype: int64

# Lag the returns

In [13]:
result[result.permno==21726].head()

,vol,beta,mktcap,momentum,short_term_reversal,52wk_high,price_rel_52wk_high,idvol,unexp_vol,turnover,...,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,ret,date,permno
243008,0.015594,0.365169,0.092574,-0.363636,-0.239195,0.406977,-0.093023,-0.095506,-0.106742,0.191584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.150000,1990-01-31,21726
243009,-0.132477,0.357143,0.092190,-0.366667,0.325137,0.385057,-0.143678,-0.093407,-0.137363,-0.102076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061765,1990-02-28,21726
243010,-0.247535,0.326087,0.091716,-0.377778,-0.252105,0.383721,-0.081395,-0.130435,-0.152174,-0.277613,...,0.014563,0.108040,-0.135922,-0.124390,0.014706,0.077143,-0.133333,0.000000,1990-03-30,21726
243011,-0.183154,0.338710,0.102156,-0.377778,0.099409,0.362069,-0.051724,-0.102151,-0.102151,-0.179726,...,0.052561,0.151685,-0.109164,-0.088556,0.060647,0.098160,-0.126316,0.063380,1990-04-30,21726
243012,-0.191687,0.434066,0.103229,-0.181818,-0.355698,0.311765,-0.041176,-0.104396,-0.192308,-0.187531,...,-0.041950,0.096389,-0.178015,-0.162837,0.018328,0.015982,-0.139053,0.074172,1990-05-31,21726


In [14]:
def lag_returns(group):
    return group['ret'].shift(-1)


result['lagged_ret'] = result.groupby('permno').apply(lag_returns).reset_index(level=0, drop=True)


/var/folders/vf/gw1t6f4j3mv84kp8kpw4j_bw0000gn/T/ipykernel_13606/1478322671.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result['lagged_ret'] = result.groupby('permno').apply(lag_returns).reset_index(level=0, drop=True)


In [15]:
result[result.permno==21726].head()


,vol,beta,mktcap,momentum,short_term_reversal,52wk_high,price_rel_52wk_high,idvol,unexp_vol,turnover,...,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,ret,date,permno,lagged_ret
243008,0.015594,0.365169,0.092574,-0.363636,-0.239195,0.406977,-0.093023,-0.095506,-0.106742,0.191584,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.150000,1990-01-31,21726,0.061765
243009,-0.132477,0.357143,0.092190,-0.366667,0.325137,0.385057,-0.143678,-0.093407,-0.137363,-0.102076,...,NaN,NaN,NaN,NaN,NaN,NaN,0.061765,1990-02-28,21726,0.000000
243010,-0.247535,0.326087,0.091716,-0.377778,-0.252105,0.383721,-0.081395,-0.130435,-0.152174,-0.277613,...,0.108040,-0.135922,-0.124390,0.014706,0.077143,-0.133333,0.000000,1990-03-30,21726,0.063380
243011,-0.183154,0.338710,0.102156,-0.377778,0.099409,0.362069,-0.051724,-0.102151,-0.102151,-0.179726,...,0.151685,-0.109164,-0.088556,0.060647,0.098160,-0.126316,0.063380,1990-04-30,21726,0.074172
243012,-0.191687,0.434066,0.103229,-0.181818,-0.355698,0.311765,-0.041176,-0.104396,-0.192308,-0.187531,...,0.096389,-0.178015,-0.162837,0.018328,0.015982,-0.139053,0.074172,1990-05-31,21726,-0.043478


In [16]:
result.drop(columns=['ret'],inplace=True)
# result = result.rename(columns={''})
y_column = ['lagged_ret']

In [17]:
result = result[~np.any(np.isnan(result), axis=1)]

In [18]:
len(result)

1045202

In [19]:
crsp_features = result.set_index(["permno", "date"])
crsp_features.head()

,,vol,beta,mktcap,momentum,short_term_reversal,52wk_high,price_rel_52wk_high,idvol,unexp_vol,turnover,...,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq,lagged_ret
permno,date,,,,,,,,,,,,,,,,,,,,,
21726,1990-03-30,-0.247535,0.326087,0.091716,-0.377778,-0.252105,0.383721,-0.081395,-0.130435,-0.152174,-0.277613,...,0.004854,0.095122,0.014563,0.108040,-0.135922,-0.124390,0.014706,0.077143,-0.133333,0.063380
11641,1990-03-30,0.130178,0.380435,-0.376233,-0.233333,0.451213,0.209302,-0.325581,0.032609,-0.010870,0.289448,...,-0.492718,-0.378049,-0.305825,-0.389447,0.145631,-0.300000,-0.487745,-0.454286,-0.023810,-0.074074
11539,1990-03-30,-0.131164,-0.413043,-0.293393,0.100000,0.039624,-0.058140,-0.372093,0.413043,0.021739,0.159270,...,-0.169903,-0.329268,-0.160194,-0.412060,-0.495146,0.104878,-0.147059,-0.162857,-0.309524,-0.018868
15456,1990-03-30,0.389053,0.423913,0.373274,-0.333333,-0.135958,0.180233,0.325581,0.097826,0.000000,0.229290,...,0.475728,0.485366,0.461165,0.494975,0.378641,0.417073,0.480392,0.488571,0.438095,-0.037307
21726,1990-04-30,-0.183154,0.338710,0.102156,-0.377778,0.099409,0.362069,-0.051724,-0.102151,-0.102151,-0.179726,...,0.063003,0.129032,0.052561,0.151685,-0.109164,-0.088556,0.060647,0.098160,-0.126316,0.074172


In [20]:
crsp_features = crsp_features[crsp_features.index.get_level_values(1)>=pd.to_datetime('1994-01-01')]

In [21]:
crsp_features_train = crsp_features.loc[~(crsp_features.index.get_level_values(0).isin(test_stocks))]
crsp_features_test = crsp_features.loc[crsp_features.index.get_level_values(0).isin(test_stocks)]

In [27]:
crsp_features_train.index.get_level_values(0).unique()

Index([11334, 24643, 64151, 63845, 15580, 76868, 14593, 10043, 77279, 77347,
       ...
       23506, 19628, 23623, 23610, 23625, 16990, 21114, 23628, 91664, 22781],
      dtype='int64', name='permno', length=8167)

In [28]:
crsp_features_test.index.get_level_values(0).unique()

Index([10516, 77520, 11691, 49330, 14702, 61241, 21020, 13769, 18075, 75846,
       ...
       23514, 20406, 23515, 21907, 23433, 20884, 20318, 92726, 23568, 22343],
      dtype='int64', name='permno', length=780)

In [24]:
y = crsp_features_train.loc[:,y_column]
y.head()

,,lagged_ret
permno,date,
11334,1994-01-31,0.031915
24643,1994-01-31,-0.048973
64151,1994-01-31,0.068966
63845,1994-01-31,-0.029762
15580,1994-01-31,0.000000


In [29]:
x_columns

['vol',
 'beta',
 'mktcap',
 'momentum',
 'short_term_reversal',
 '52wk_high',
 'price_rel_52wk_high',
 'idvol',
 'unexp_vol',
 'turnover',
 'mkt_to_book',
 'revtq',
 'niq',
 'atq',
 'epsfxq',
 'cheq',
 'dlttq',
 'cogsq',
 'oibdpq',
 'cshoq']

In [30]:
X = crsp_features_train.loc[:,x_columns]
X.head()

,,vol,beta,mktcap,momentum,short_term_reversal,52wk_high,price_rel_52wk_high,idvol,unexp_vol,turnover,mkt_to_book,revtq,niq,atq,epsfxq,cheq,dlttq,cogsq,oibdpq,cshoq
permno,date,,,,,,,,,,,,,,,,,,,,
11334,1994-01-31,0.249803,-0.310811,-0.022423,-0.075103,0.040833,-0.155804,0.003311,0.099099,0.041289,0.268686,-0.322242,0.209099,0.245479,0.208826,-0.042539,0.031236,0.253300,0.225916,0.300491,0.336778
24643,1994-01-31,0.451220,-0.046847,0.448072,-0.202343,0.052842,0.396130,0.129648,-0.204505,-0.449183,0.312746,0.269371,0.450495,0.360307,0.438581,0.168790,0.447560,0.425808,0.456581,0.428256,0.394137
64151,1994-01-31,0.180960,-0.203153,-0.087333,-0.184887,-0.090472,-0.334012,-0.106979,0.204955,0.009982,-0.118411,-0.365087,0.235502,0.263110,0.197452,-0.015696,0.125627,0.332044,0.222298,0.281818,0.289119
63845,1994-01-31,0.269866,-0.008559,0.128639,-0.357602,-0.231385,0.009420,-0.084310,-0.151351,-0.356624,0.079072,-0.329991,0.311410,-0.405967,0.344404,-0.356005,0.458504,0.200956,0.348485,0.151106,0.290806
15580,1994-01-31,-0.097168,0.463964,-0.246263,-0.273542,0.320456,-0.413187,0.398625,0.358559,0.225045,-0.272620,-0.362808,0.001179,0.001808,0.030027,-0.213376,0.138851,0.076240,0.057214,-0.038575,0.143189


In [31]:
importlib.reload(ipca)

<module 'util.ipca' from '/Users/jandh/Desktop/Old Desktop/od/2 Quarter/Flexible/Factor-Research/notebooks/../src/util/ipca.py'>

In [32]:
model = ipca.InstrumentedPCA(n_factors=5, intercept=True)
model = model.fit(X = X,y=y)
Gamma, Factors = model.get_factors(label_ind=True)
    

The panel dimensions are:
n_samples: 8167 , L: 20 , T: 371


[========================================================================] 100%


Step 1 - Aggregate Update: 1.5901354574476962
Step 2 - Aggregate Update: 1.2579661650643335
Step 3 - Aggregate Update: 0.1531666312079718
Step 4 - Aggregate Update: 1.159515859590389
Step 5 - Aggregate Update: 1.0996267772457713
Step 6 - Aggregate Update: 0.038541439364856656
Step 7 - Aggregate Update: 0.028462316288177573
Step 8 - Aggregate Update: 0.02379137995098568
Step 9 - Aggregate Update: 0.01945640720113634
Step 10 - Aggregate Update: 0.01586356780835657
Step 11 - Aggregate Update: 0.0130855278100378
Step 12 - Aggregate Update: 0.011097708843278875
Step 13 - Aggregate Update: 0.014431891952993159
Step 14 - Aggregate Update: 0.020866668858103288
Step 15 - Aggregate Update: 0.02810997803098947
Step 16 - Aggregate Update: 0.036091602366621706
Step 17 - Aggregate Update: 0.044283495936942446
Step 18 - Aggregate Update: 0.051491213655736195
Step 19 - Aggregate Update: 0.05589603128075411
Step 20 - Aggregate Update: 0.09279973001093668
Step 21 - Aggregate Update: 1.3389620786132048
S

# Cross Sectional $R^2$ test

In [33]:
Gamma

,0,1,2,3,4,5
vol,0.022731,-0.050899,-0.348040,-0.015142,-0.313735,-0.019429
beta,0.070689,0.091986,0.014618,-0.040348,0.420520,-0.010485
mktcap,0.108563,0.404305,0.300605,0.452745,0.168832,0.003157
momentum,-0.033205,-0.006704,0.109470,0.508216,-0.165465,0.002436
short_term_reversal,0.013601,0.017901,-0.066427,-0.097227,0.137132,0.002451
52wk_high,-0.100976,-0.340417,-0.363407,0.078982,0.453260,-0.002051
price_rel_52wk_high,-0.065829,-0.171203,-0.082220,0.483006,-0.074305,-0.009591
idvol,0.115971,0.080412,-0.107283,0.084305,-0.067501,-0.003471
unexp_vol,-0.043464,-0.070163,-0.146177,0.088756,-0.325104,0.012374
turnover,0.112865,0.142499,0.021549,0.066397,0.238924,-0.001303


In [34]:
Factors

,1994-01-31,1994-02-28,1994-03-31,1994-04-29,1994-05-31,1994-06-30,1994-07-29,1994-08-31,1994-09-30,1994-10-31,...,2024-02-29,2024-03-28,2024-04-30,2024-05-31,2024-06-28,2024-07-31,2024-08-30,2024-09-30,2024-10-31,2024-11-29
0,-0.124194,-0.211738,-0.119948,-0.070955,-0.330786,0.174370,0.321069,0.003888,-0.110312,-0.295531,...,0.167263,-0.152600,0.287733,-0.227219,0.259671,-0.009509,0.056979,0.015807,0.067654,-0.161346
1,0.221278,0.075449,0.209350,-0.067586,0.133767,0.044029,0.007154,0.073907,0.253129,0.172982,...,-0.013825,-0.272215,-0.034047,0.025992,0.187233,-0.058789,0.041448,-0.021193,0.494519,0.188661
2,-0.005477,-0.022209,0.100942,0.022905,0.035555,0.048122,0.055119,-0.059343,-0.042077,-0.061657,...,0.142916,0.069527,0.036433,0.011876,0.078765,0.105391,0.046126,-0.040666,-0.005812,-0.155677
3,0.009664,-0.032599,0.040988,-0.004761,-0.003322,0.019393,0.024138,0.042729,0.042203,0.028489,...,-0.021021,-0.061691,-0.041059,0.002477,-0.001380,0.033776,-0.014586,0.044191,0.099270,-0.064947
4,0.003596,0.017943,-0.004087,0.015529,0.011471,0.004861,0.022370,-0.027097,0.039806,0.031898,...,-0.043212,-0.019636,0.020916,0.041192,0.057288,0.011327,0.023427,-0.001647,0.029967,-0.080812
5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
X_test = crsp_features_test.loc[:,x_columns]
y_test = crsp_features_test.loc[:,y_column]

In [37]:
model.score(X_test,y_test)

0.05599678742794456

In [38]:
model.score(X,y)

0.08082851320629758

In [39]:
Gamma.to_csv('../data/test_model_gamma.csv')

In [40]:
Factors.to_csv('../data/test_model_factors.csv')

In [688]:
# csdf_test = pd.DataFrame()
# X_test = crsp_features_test.loc[:,x_columns]
# # X_test = X_test.fillna(0)  ### This maybe an issue
# for i in range(1,len(Factors.columns)):
#     cs_returns = pd.DataFrame(X_test[X_test.index.get_level_values(1)==Factors.columns[i]].dot(Gamma).dot(Factors.iloc[:,1]),columns=["pred"])
#     # cs_returns
#     csdf_test = pd.concat([csdf_test, cs_returns], axis=0)

# y_test = crsp_features_test.loc[:,y_column]
# combined_df = csdf_test.merge(y_test,left_on=['permno', 'date'],right_on=['permno', 'date'],how='inner')
# combined_df.dropna(inplace=True)
# r2_score(combined_df['lagged_ret'],combined_df['pred'])

-0.011934333073995829

In [693]:
factor_models = {}

In [702]:
score_df = pd.DataFrame(columns=['out_of_sample','in_sample'])
for n_factors in range(1,15):
    model = ipca.InstrumentedPCA(n_factors=n_factors, intercept=False)
    model = model.fit(X = X,y=y)
    Gamma, Factors = model.get_factors(label_ind=True)
    oos_score = model.score(X_test,y_test)
    ins_score = model.score(X,y)
    score_df.loc[n_factors,:] = [oos_score,ins_score]
    # factor_models[n_factors] = {"Gamma": Gamma,"Factors": Factors}
    

The panel dimensions are:
n_samples: 8167 , L: 20 , T: 371


[========================================================================] 100%


Step 1 - Aggregate Update: 1.5887358232260023
Step 2 - Aggregate Update: 0.7646207288785895
Step 3 - Aggregate Update: 0.15189468958991947
Step 4 - Aggregate Update: 0.03822051129011039
Step 5 - Aggregate Update: 0.009421476625537162
Step 6 - Aggregate Update: 0.002245880892324248
Step 7 - Aggregate Update: 0.0005300881683640846
Step 8 - Aggregate Update: 0.00012456996101906093
Step 9 - Aggregate Update: 2.9202885002854728e-05
Step 10 - Aggregate Update: 6.836061509973845e-06
-- Convergence Reached --


The panel dimensions are:
n_samples: 8167 , L: 20 , T: 371


[========================================================================] 100%


...

[========================================================================] 100%


Step 1 - Aggregate Update: 1.3154036802627187
Step 2 - Aggregate Update: 1.2383879328305634
Step 3 - Aggregate Update: 0.9868936994143676
Step 4 - Aggregate Update: 0.36687223998099505
Step 5 - Aggregate Update: 0.9819405960252771
Step 6 - Aggregate Update: 0.1318982343345526
Step 7 - Aggregate Update: 0.12903557458645842
Step 8 - Aggregate Update: 0.16949045597982146
Step 9 - Aggregate Update: 0.1273694389661373
Step 10 - Aggregate Update: 1.321114429133239
Step 11 - Aggregate Update: 0.055783219377165696
Step 12 - Aggregate Update: 0.04552058360999442
Step 13 - Aggregate Update: 0.03626818807471699
Step 14 - Aggregate Update: 0.028681765891084822
Step 15 - Aggregate Update: 0.02269548958367132
Step 16 - Aggregate Update: 0.01803023550497756
Step 17 - Aggregate Update: 0.01439642133525279
Step 18 - Aggregate Update: 0.01155318786594011
Step 19 - Aggregate Update: 0.009493518692585745
Step 20 - Aggregate Update: 0.009231470797259389
Step 21 - Aggregate Update: 0.011196928114263038
Step

The panel dimensions are:
n_samples: 8167 , L: 20 , T: 371


[========================================================================] 100%


Step 1 - Aggregate Update: 1.2112060720475801
Step 2 - Aggregate Update: 1.393582000082315
Step 3 - Aggregate Update: 0.949546604033823
Step 4 - Aggregate Update: 0.24872917264886474
Step 5 - Aggregate Update: 0.20050103753493775
Step 6 - Aggregate Update: 0.1289172475134972
Step 7 - Aggregate Update: 0.1190798742369236
Step 8 - Aggregate Update: 0.0909854503873457
Step 9 - Aggregate Update: 0.06142564848002914
Step 10 - Aggregate Update: 0.04197795121164416
Step 11 - Aggregate Update: 1.1184797829051738
Step 12 - Aggregate Update: 0.03866246263285711
Step 13 - Aggregate Update: 0.04063997785340917
Step 14 - Aggregate Update: 0.038634298911445486
Step 15 - Aggregate Update: 0.033294160542607754
Step 16 - Aggregate Update: 0.02698037818538368
Step 17 - Aggregate Update: 0.021139326165773964
Step 18 - Aggregate Update: 0.016303801563326836
Step 19 - Aggregate Update: 0.012826755097232856
Step 20 - Aggregate Update: 0.010165092649372931
Step 21 - Aggregate Update: 0.008085413239225292
Ste

In [703]:
score_df

,out_of_sample,in_sample
1,0.029764,0.050813
2,0.043129,0.063624
3,0.04859,0.072159
4,0.051101,0.077109
5,0.055128,0.080087
6,0.056739,0.082846
7,0.057463,0.085058
8,0.058569,0.086811
9,0.059538,0.088444
10,0.060082,0.08988


In [705]:
len(X.columns)

20